Importar librerías necesarias

In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, accuracy_score


Cargar dataset OCR

In [2]:
df = pd.read_csv(r"D:\MAXITEL\Escritorio\ginger\MAESTRIA\INTELIGENCIA ARTIFICIAL\proyecto-ml-telecom\Supervisado\ocrdataset.csv")

# Eliminar filas con valores NaN
df = df.dropna()

print("Tamaño del dataset después de eliminar NaN:", df.shape)
print("Columnas disponibles:", df.columns)
df.head()

Tamaño del dataset después de eliminar NaN: (585, 21)
Columnas disponibles: Index(['Tx', 'Rx', 'Various Points', 'SNR Receiver', 'SNR Stages',
       'BER Receiver', 'BER Environmental', 'Modulation Format',
       'Modulation Depth', 'Transmission Distance', 'Distance Between',
       'Fiber Attenuation', 'Splice Losses', 'Optical Amplifier Gain',
       'PMD Coefficient', 'PMD Compensation', 'CD Coefficient',
       'CD Compensation', 'Temperature', 'Humidity', 'Signal Quality'],
      dtype='object')


,Tx,Rx,Various Points,SNR Receiver,SNR Stages,BER Receiver,BER Environmental,Modulation Format,Modulation Depth,Transmission Distance,...,Fiber Attenuation,Splice Losses,Optical Amplifier Gain,PMD Coefficient,PMD Compensation,CD Coefficient,CD Compensation,Temperature,Humidity,Signal Quality
0,10.5,15.2,12.8,23.6,22.1,0.000001,0.000002,AM,0.8,20,...,0.10,0.20,10,0.05,PMD Compensated,0.03,CD Compensated,25.0,60.0,Good
1,11.2,16.5,12.6,24.2,22.8,0.000001,0.000002,FM,0.7,21,...,0.11,0.22,12,0.06,PMD Compensated,0.04,CD Compensated,26.0,61.0,Good
2,9.8,14.8,12.9,23.4,21.9,0.000001,0.000001,PM,0.9,19,...,0.09,0.18,9,0.04,PMD Compensated,0.02,CD Compensated,24.0,59.0,Good
3,12.1,18.2,13.2,24.8,23.5,0.000002,0.000002,AM,0.8,22,...,0.12,0.24,11,0.07,PMD Compensated,0.05,CD Compensated,27.0,62.0,Good
4,10.9,15.8,13.0,24.0,22.5,0.000001,0.000002,FM,0.7,20,...,0.10,0.20,10,0.05,PMD Compensated,0.03,CD Compensated,25.0,60.0,Good


Separar características y etiquetas

In [3]:
# Ajusta "Signal Quality" si tu dataset tiene otro nombre
target_col = "Signal Quality"  
X = df.drop(target_col, axis=1)
y = df[target_col]

# División en train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Selección de características con Random Forest

In [7]:
# ...existing code...
# Codifica variables categóricas antes de usar RandomForest
X_train_enc = pd.get_dummies(X_train)
X_test_enc = pd.get_dummies(X_test)

# Alinea columnas para que ambos tengan las mismas características
X_train_enc, X_test_enc = X_train_enc.align(X_test_enc, join='left', axis=1, fill_value=0)

# Ahora puedes entrenar el selector
rf_selector = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selector.fit(X_train_enc, y_train)

selector = SelectFromModel(rf_selector, prefit=True, threshold="median")
X_train_selected = selector.transform(X_train_enc)
X_test_selected = selector.transform(X_test_enc)

print("Características originales:", X_train_enc.shape[1])
print("Características seleccionadas:", X_train_selected.shape[1])
# ...existing code...

Características originales: 22
Características seleccionadas: 11


C:\Users\cris2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\cris2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Selección de hiperparámetros con GridSearchCV

In [8]:
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring="accuracy"
)

grid_search.fit(X_train_selected, y_train)
print("Mejores hiperparámetros:", grid_search.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Mejores hiperparámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


Evaluación del modelo

In [9]:
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_selected)

print("Accuracy en test:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy en test: 1.0

Reporte de clasificación:
               precision    recall  f1-score   support

        Good       1.00      1.00      1.00       108
        Poor       1.00      1.00      1.00         9

    accuracy                           1.00       117
   macro avg       1.00      1.00      1.00       117
weighted avg       1.00      1.00      1.00       117

